In [1]:
# !pip install pandas-profiling

In [2]:
# Import our modules
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline

# current version of seaborn generates a bunch of warnings that we'll ignore
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

#import missingno as msno # missing data visualization module for Python
import pandas_profiling

import gc
import datetime
from datetime import datetime

# Using ggplot's style
plt.style.use('ggplot')

## **Context**

Sebagai seorang data scientist di suatu e-commerce, penulis dimintai bantuan oleh tim marketing untuk membantu mereka dalam mengatur strategi pemasaran perusahaan. Mereka ingin mengetahui siapa pelanggan - pelanggan terbaik mereka, siapa yang berpotensial untuk menjadi pelanggan terbaik mereka dan siapa yang merupakan pelanggan-pelanggan yang kurang memberikan nilai kepada perusahaan. 

Salah satu cara yang bisa dilakukan untuk memahami pelanggan penulis adalah dengan melakukan segmentasi pelanggan. Segmentasi adalah proses pengelompokan pelanggan menjadi beberapa kelompok berdasarkan karakteristik mereka. Kita dapat menggunakan banyak variabel untuk membuat segmentasi pelanggan kita. Informasi seperti demografis pelanggan , geografis, psikografis, teknografis, dan perilaku sering digunakan sebagai pembeda untuk mensegmentasi pelanggan.

Kali ini penulis hanya akan melakukan segmentasi berdasarkan perilaku transaksi pelanggan. Hal ini menyesuaikan dengan data set yang sudah di siapkan tim marketing yaitu data transaksi di platform penulis selama satu tahun terakhir. Kamu akan menggunakan data Recency, Frequency dan Monetary yang sudah terbukti untuk merepresentasikan aktivitas transaksi pelanggan di berbagai industri. 

In [3]:
df_ecommerce = pd.read_excel('data_ecommerce.xlsx', sheet_name='ecommerce_data') # 541.909 rows × 8 columns
df_customer = pd.read_excel('data_ecommerce.xlsx', sheet_name='customer_data') # 4.372 rows × 9 columns

# **Data Preparation**
**Data Collection**

In [4]:
display(print(df_ecommerce))
display(print(df_customer))

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   White Hanging Heart T-Light Holder         6   
1         536365     71053                  White Metal Lantern         6   
2         536365    84406B       Cream Cupid Hearts Coat Hanger         8   
3         536365    84029G  Knitted Union Flag Hot Water Bottle         6   
4         536365    84029E       Red Woolly Hottie White Heart.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          Pack Of 20 Spaceboy Napkins        12   
541905    581587     22899         Children'S Apron Dolly Girl          6   
541906    581587     23254        Childrens Cutlery Dolly Girl          4   
541907    581587     23255      Childrens Cutlery Circus Parade         4   
541908    581587     22138        Baking Set 9 Piece Retrospot          3   

                InvoiceDate  UnitPrice  CustomerID         Country  
0     

None

      CustomerID       CustomerName                 CustomerEmail  Gender  \
0          12346       Loleta Varga         lvarga0@wikipedia.org  Female   
1          12347   Gerrie Dell Casa         gdellcasa1@disqus.com  Female   
2          12348  Barty MacDiarmond      bmacdiarmond2@tumblr.com    Male   
3          12349   Darrelle Bissell           dbissell3@yandex.ru  Female   
4          12350       Keelia Gange          kgange4@symantec.com  Female   
...          ...                ...                           ...     ...   
4367       18280          Ora Meads       omeadsa7@techcrunch.com  Female   
4368       18281          Gage Call    gcalla8@huffingtonpost.com    Male   
4369       18282     Mela Sparkwell  msparkwella9@yellowpages.com  Female   
4370       18283       Bruno Struan            bstruanaa@youtu.be    Male   
4371       18287     Juline Dowding            jdowdingab@php.net  Female   

     DateOfBirth        Profession  MonthlyIncome         EducationalLevel 

None

**Database connection**

In [5]:
df = pd.merge(df_ecommerce, df_customer, left_on='CustomerID', right_on='CustomerID', how='left')
df.head() # 541.909 rows × 16 columns

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,CustomerEmail,Gender,DateOfBirth,Profession,MonthlyIncome,EducationalLevel,MaritalStatus
0,536365,85123A,White Hanging Heart T-Light Holder,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.9,Have Bachelor Degree,Single
1,536365,71053,White Metal Lantern,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.9,Have Bachelor Degree,Single
2,536365,84406B,Cream Cupid Hearts Coat Hanger,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.9,Have Bachelor Degree,Single
3,536365,84029G,Knitted Union Flag Hot Water Bottle,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.9,Have Bachelor Degree,Single
4,536365,84029E,Red Woolly Hottie White Heart.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.9,Have Bachelor Degree,Single


**Data Understanding / Simple-Exploration**

In [6]:
# !pip install pandasql
import pandasql

*1. Count the number of customer per country*

In [7]:
query = """
        select Country,
        count(distinct CustomerID) 
        from df
        group by Country
        order by count(distinct CustomerID) DESC;
        """
df_cc = pandasql.sqldf(query)
df_cc.head()

,Country,count(distinct CustomerID)
0,United Kingdom,3950
1,Germany,95
2,France,87
3,Spain,31
4,Belgium,25


*2. Count the number of order per country*

In [8]:
query = """
        select Country,
        count(Quantity) as Quantity
        from df
        group by Country
        order by Quantity DESC; 
        """
df_oc = pandasql.sqldf(query)
df_oc

,Country,Quantity
0,United Kingdom,495478
1,Germany,9495
2,France,8557
3,EIRE,8196
4,Spain,2533
5,Netherlands,2371
6,Belgium,2069
7,Switzerland,2002
8,Portugal,1519
9,Australia,1259


> *2. Count  5 number of order per city*

In [9]:
query = """
        select Country,
        count(Quantity) as Quantity
        from df
        group by Country
        order by Quantity DESC
        limit 5; 
        """
df_oc5 = pandasql.sqldf(query)
df_oc5

,Country,Quantity
0,United Kingdom,495478
1,Germany,9495
2,France,8557
3,EIRE,8196
4,Spain,2533


*3. Find the first order date of each customer*

In [10]:
query = """
        select CustomerID,
        min(InvoiceDate) as Date
        from df
        group by CustomerID; 
        """
df_fod = pandasql.sqldf(query)
df_fod

,CustomerID,Date
0,NaN,1/13/2011 10:11
1,12346.0,1/18/2011 10:01
2,12347.0,1/26/2011 14:30
3,12348.0,1/25/2011 10:42
4,12349.0,11/21/2011 9:51
...,...,...
4368,18280.0,2011-07-03 09:52:00
4369,18281.0,2011-12-06 10:53:00
4370,18282.0,2011-02-12 11:43:00
4371,18283.0,1/23/2011 14:58


*4. Top 10 transactions CustomerID 17850.0 - Janis Sparwell*

In [11]:
query = """
        select
            StockCode, InvoiceNo, Description as Product, Quantity as jumlah_quantity,
            InvoiceDate as tanggal_transaksi
        from 
            df
        where
            CustomerID = 17850.0
        group by Product 
        order by jumlah_quantity desc
        limit 10; 
        """
df_noc = pandasql.sqldf(query)
df_noc

,StockCode,InvoiceNo,Product,jumlah_quantity,tanggal_transaksi
0,84406B,536365,Cream Cupid Hearts Coat Hanger,8,2010-01-12 08:26:00
1,82482,536373,Wooden Picture Frame White Finish,6,2010-01-12 09:02:00
2,82494L,536373,Wooden Frame Antique White,6,2010-01-12 09:02:00
3,71053,536365,White Metal Lantern,6,2010-01-12 08:26:00
4,85123A,536365,White Hanging Heart T-Light Holder,6,2010-01-12 08:26:00
5,21068,536373,Vintage Billboard Love/Hate Mug,6,2010-01-12 09:02:00
6,21071,536373,Vintage Billboard Drink Me Mug,6,2010-01-12 09:02:00
7,21871,536373,Save The Planet Mug,6,2010-01-12 09:02:00
8,37370,536373,Retro Coffee Mugs Assorted,6,2010-01-12 09:02:00
9,84029E,536365,Red Woolly Hottie White Heart.,6,2010-01-12 08:26:00


# **Data Quality Check and Remediation**

**Missing Values**

In [12]:
# check missing values for each column 
df.isnull().sum().sort_values(ascending=False)

CustomerID          135080
CustomerName        135080
CustomerEmail       135080
Gender              135080
DateOfBirth         135080
Profession          135080
MonthlyIncome       135080
EducationalLevel    135080
MaritalStatus       135080
Description           1454
InvoiceNo                0
StockCode                0
Quantity                 0
InvoiceDate              0
UnitPrice                0
Country                  0
dtype: int64

In [13]:
# check out the rows with missing values
df[df.isnull().any(axis=1)].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,CustomerEmail,Gender,DateOfBirth,Profession,MonthlyIncome,EducationalLevel,MaritalStatus
622,536414,22139,NaN,56,2010-01-12 11:52:00,0.00,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,536544,21773,Decorative Rose Bathroom Bottle,1,2010-01-12 14:32:00,2.51,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,536544,21774,Decorative Cats Bathroom Bottle,2,2010-01-12 14:32:00,2.51,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,536544,21786,Polkadot Rain Hat,4,2010-01-12 14:32:00,0.85,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,536544,21787,Rain Poncho Retrospot,2,2010-01-12 14:32:00,1.66,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1447,536544,21790,Vintage Snap Cards,9,2010-01-12 14:32:00,1.66,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1448,536544,21791,Vintage Heads And Tails Card Game,2,2010-01-12 14:32:00,2.51,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1449,536544,21801,Christmas Tree Decoration With Bell,10,2010-01-12 14:32:00,0.43,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1450,536544,21802,Christmas Tree Heart Decoration,9,2010-01-12 14:32:00,0.43,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451,536544,21803,Christmas Tree Star Decoration,11,2010-01-12 14:32:00,0.43,NaN,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# df_new without missing values
df_new = df.dropna()
df_new

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,CustomerEmail,Gender,DateOfBirth,Profession,MonthlyIncome,EducationalLevel,MaritalStatus
0,536365,85123A,White Hanging Heart T-Light Holder,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.90,Have Bachelor Degree,Single
1,536365,71053,White Metal Lantern,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.90,Have Bachelor Degree,Single
2,536365,84406B,Cream Cupid Hearts Coat Hanger,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.90,Have Bachelor Degree,Single
3,536365,84029G,Knitted Union Flag Hot Water Bottle,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.90,Have Bachelor Degree,Single
4,536365,84029E,Red Woolly Hottie White Heart.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,Janis Sparwell,jsparwell1c@seattletimes.com,Female,18/09/1973,Accountant,1363949.90,Have Bachelor Degree,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,Pack Of 20 Spaceboy Napkins,12,2011-09-12 12:50:00,0.85,12680.0,France,Dulce Widdall,dwiddall7n@go.com,Female,27/03/1957,Dental Hygienist,6346645.44,Have Bachelor Degree,Married
541905,581587,22899,Children'S Apron Dolly Girl,6,2011-09-12 12:50:00,2.10,12680.0,France,Dulce Widdall,dwiddall7n@go.com,Female,27/03/1957,Dental Hygienist,6346645.44,Have Bachelor Degree,Married
541906,581587,23254,Childrens Cutlery Dolly Girl,4,2011-09-12 12:50:00,4.15,12680.0,France,Dulce Widdall,dwiddall7n@go.com,Female,27/03/1957,Dental Hygienist,6346645.44,Have Bachelor Degree,Married
541907,581587,23255,Childrens Cutlery Circus Parade,4,2011-09-12 12:50:00,4.15,12680.0,France,Dulce Widdall,dwiddall7n@go.com,Female,27/03/1957,Dental Hygienist,6346645.44,Have Bachelor Degree,Married


In [18]:
# check missing values for each column 
df_new.isnull().sum().sort_values(ascending=False)

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID          0
Country             0
CustomerName        0
CustomerEmail       0
Gender              0
DateOfBirth         0
Profession          0
MonthlyIncome       0
EducationalLevel    0
MaritalStatus       0
dtype: int64

Dari hasil info tersebut beberapa hal yang dapat di simpulkan adalah
- Sudah tidak ada null value di data kita.
- Namun kita perlu merubah beberpa tipe data kedalam bentuk yang semestinya. Contoh: InvoiceDate dari object menjadi tipe date and time dan juga merubah CustomerID menjadi string serta feature atau columns lainnya.

**Data Anomalies**

Dalam memahami anomali pada data sebaiknya harus dapat memahami tipe data terlebih dahulu. Dalam melihat informasi tersebut kita dapat menggunakan fungsi `info()`. Sehingga dengan mudah kita bisa mengetahui jumlah baris dan kolom, nama kolom, identifikasi null values,  dan tipe data. 

In [19]:
# check datatype from the data
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   InvoiceNo         406829 non-null  object 
 1   StockCode         406829 non-null  object 
 2   Description       406829 non-null  object 
 3   Quantity          406829 non-null  int64  
 4   InvoiceDate       406829 non-null  object 
 5   UnitPrice         406829 non-null  float64
 6   CustomerID        406829 non-null  float64
 7   Country           406829 non-null  object 
 8   CustomerName      406829 non-null  object 
 9   CustomerEmail     406829 non-null  object 
 10  Gender            406829 non-null  object 
 11  DateOfBirth       406829 non-null  object 
 12  Profession        406829 non-null  object 
 13  MonthlyIncome     406829 non-null  float64
 14  EducationalLevel  406829 non-null  object 
 15  MaritalStatus     406829 non-null  object 
dtypes: float64(3), int64

**Data Information**

In [20]:
# df_new.describe()
# df.describe(exclude=[np.number]).round(2)

# Describing a numeric Series
display(df_new.describe().round(2))

# Excluding numeric columns from a DataFrame description
display(df_new.describe(exclude=[np.number]).round(2))

,Quantity,UnitPrice,CustomerID,MonthlyIncome
count,406829.00,406829.00,406829.00,406829.00
mean,12.06,3.46,15287.69,4349443.53
std,248.69,69.32,1713.60,2573420.03
min,-80995.00,0.00,12346.00,5222.45
25%,2.00,1.25,13953.00,2173278.17
50%,5.00,1.95,15152.00,4334768.12
75%,12.00,3.75,16791.00,6470575.04
max,80995.00,38970.00,18287.00,8983494.72


,InvoiceNo,StockCode,Description,InvoiceDate,Country,CustomerName,CustomerEmail,Gender,DateOfBirth,Profession,EducationalLevel,MaritalStatus
count,406829,406829,406829,406829,406829,406829,406829,406829,406829,406829,406829,406829
unique,22190,3684,3896,20460,37,4372,4372,2,3830,34,2,4
top,576339,85123A,White Hanging Heart T-Light Holder,11/14/2011 15:27,United Kingdom,Drusy Comer,dcomer16@oaic.gov.au,Female,17/12/1972,Student,Haven't Bachelor Degree,Single
freq,542,2077,2070,543,361878,7983,7983,219398,7983,62388,224403,187217


# **Data Wrangling**

Berikut ini adalah **Data Wrangling Flow** atau tahapan-tahapan dalam membersihkan data.
* Pertama, **Data cleaning**: Mendiagnosis dan menangani masalah kualitas data
* Kedua, **Data integration**: Menggabungkan data dari berbagai sumber untuk membentuk dataset yang lengkap dan kaya akan informasi
* Ketiga, **Data enrichment**: Memperkaya dataset dengan feature generation, transformasi data (`data transformation`), agregasi data, penyederhanaan data, dan
lainnya.
* Terakhir, **Data storing**: adalah menyimpan data yang sudah bersih.

In [21]:
# change the invoice_date format - String to Timestamp format
# Change the start time column to datetime datatype
df_new['InvoiceDate'] =  pd.to_datetime(df_new['InvoiceDate'])

# Extract the desired value (day-month-year hour-minute-second) using .dt.strftime('') 
df_new['InvoiceDate'] = df_new['InvoiceDate'].dt.strftime('%d-%m-%Y %H:%M:%S')

In [22]:
# change columns data type - String to Int type 
df_new['CustomerID'] = df_new['CustomerID'].astype('int64')

In [23]:
# remove negative quanitites
df_new = df_new[df_new.Quantity > 0]

In [24]:
# add new feature - Amount spent
df_new['AmountSpent'] = df_new['Quantity'] * df_new['UnitPrice']


In [25]:
# change the invoice_date format - String to Timestamp format
# Change the start time column to datetime datatype
df_new['InvoiceDate'] =  pd.to_datetime(df_new['InvoiceDate'])

# Extract the desired value (days) using .dt.strftime('')
df_new['Dayname'] = df_new['InvoiceDate'].dt.strftime('%A')

# Change dayname feature data type to categorical
df_new['Dayname'] = pd.Categorical(df_new['Dayname'],
                                    ordered=True,
                                    categories=['Monday',
                                               'Tuesday',
                                               'Wednesday',
                                               'Thursday',
                                               'Friday',
                                               'Saturday',
                                               'Sunday'])

In [26]:
# Add Month, Day and Hour Colummns that we'll use for further analysis
# Starting with Month_Year First
df_new['month_year'] = df_new['InvoiceDate'].dt.to_period('M')

In [27]:
# Continue the process for adding day, hour, etc colummns
L = ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter']
df_new = df_new.join(pd.concat((getattr(df_new['InvoiceDate'].dt, i).rename(i) for i in L), axis=1))

In [28]:
# check unique from dayofweek feature
display(df_new.dayofweek.unique())

# day of week -> +1 to make Monday=1.....until Sunday=7
df_new['dayofweek'] = df_new['dayofweek'] + 1

array([2, 3, 4, 6, 0, 1], dtype=int64)

In [29]:
# Change the date of birth column to datetime datatype
df_new['DateOfBirth'] = pd.to_datetime(df_new['DateOfBirth'])

# Extract the desired value (year) using .dt.strftime('')
df_new['birth_year'] = df_new['DateOfBirth'].dt.strftime('%Y')

# change  date of birth columns data type - String to Int type 
df_new['birth_year'] = df_new['birth_year'].astype('int64')

In [30]:
# Add Age feature to the data
df_new['Age'] = 2011 - df_new['birth_year'] 

In [31]:
# Create a function to categorize generation data based on age grouping
def convert_age_category(row):
    if row['Age'] <= 22:
        return 'Generation Z'
    elif row['Age'] <= 38:
        return 'Millenials'
    elif row['Age'] <= 54:
        return 'Generation X'
    elif row['Age'] <= 73:
        return 'Baby Boomers'
    elif row['Age'] <= 91:
        return 'Silent'
    else:
        return 'unknown'

# Apply function to the data
df_new['age_category'] = df_new.apply(convert_age_category, axis=1)

In [32]:
# Change the start time column to datetime datatype
df_new['InvoiceDate'] =  pd.to_datetime(df_new['InvoiceDate'])

# Change the date of birth column to datetime datatype
df_new['DateOfBirth'] = pd.to_datetime(df_new['DateOfBirth'])

In [33]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         397924 non-null  object        
 1   StockCode         397924 non-null  object        
 2   Description       397924 non-null  object        
 3   Quantity          397924 non-null  int64         
 4   InvoiceDate       397924 non-null  datetime64[ns]
 5   UnitPrice         397924 non-null  float64       
 6   CustomerID        397924 non-null  int64         
 7   Country           397924 non-null  object        
 8   CustomerName      397924 non-null  object        
 9   CustomerEmail     397924 non-null  object        
 10  Gender            397924 non-null  object        
 11  DateOfBirth       397924 non-null  datetime64[ns]
 12  Profession        397924 non-null  object        
 13  MonthlyIncome     397924 non-null  float64       
 14  Educ

In [34]:
df_new.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,CustomerEmail,...,year,month,day,dayofweek,dayofyear,weekofyear,quarter,birth_year,Age,age_category
399884,571289,21326,Aged Glass Silver T-Light Holder,12,2011-10-16 15:27:00,0.65,15113,United Kingdom,Josefa Twycross,jtwycross14@cafepress.com,...,2011,10,16,7,289,41,4,1978,33,Millenials
146395,548987,20754,Retrospot Red Washing Up Gloves,2,2011-04-05 12:50:00,2.10,14535,United Kingdom,Sissy Iddon,siddonha@wikipedia.org,...,2011,4,5,2,95,14,2,1989,22,Generation Z
465860,576255,23314,Vintage Christmas Tablecloth,1,2011-11-14 14:09:00,12.50,15993,United Kingdom,Ogden Sterricks,osterricksja@photobucket.com,...,2011,11,14,1,318,46,4,1992,19,Generation Z
428263,573416,22381,Toy Tidy Pink Polkadot,20,2011-10-31 08:29:00,2.10,14688,United Kingdom,Josh Noice,jnoicekd@thetimes.co.uk,...,2011,10,31,1,304,44,4,1991,20,Generation Z
137053,548011,22974,Childrens Dolly Girl Mug,48,2011-03-29 11:14:00,1.45,14646,Netherlands,Janifer Comiam,jcomiamjj@angelfire.com,...,2011,3,29,2,88,13,1,1975,36,Millenials


In [35]:
# Rename the dataframe to make code more readable
df2 = df_new.copy()

In [36]:
# # export dataset to excel
# df2.to_excel('df2.xlsx',index=False)